### Figure 1C - MTC capture reproducibility across 2 biological replicates. 

In this notebook we present the steps followed to generate Figure 1C in our paper.  This figure compares the MTC-captured transcripts on a gene-by-gene basis between two biological replicates.  Other replicates and comparison of the lysate replicates can be found by looking at our supplemental figures. 

Note - this notebook makes use of bokeh's export svg functionality to create svgs of each image to inlcude in Adobe illustrator.  However each figure is also generated as a preview in Jupyter notebook.  Simply don't run cells that save the image to svg if this is an issue for you and you should still be able to preview the interactive figures. 

In [1]:
import pandas as pd
import os
from bokeh.io import push_notebook, show, output_notebook, export_svg, curdoc
from bokeh.plotting import figure
from bokeh.models import LinearColorMapper, ColorBar
import datetime
from scipy.stats import gaussian_kde
import numpy as np
from sklearn.metrics import r2_score
from bokeh.themes import Theme
curdoc().theme = Theme(filename="../../figure_theme.yaml")

output_notebook()

Loading BokehJS ...

First we will fetch the correct reads for this experiment.  We will fetch all reads from the experiment folder, though we only use two for comparison in the figure. 

In [2]:
def normalization(gene_counts):
    '''Convert gene_counts into RPM after removing capsule reads and discarding reads with < 100 reads.'''
    sample_df = gene_counts.copy()
    sample_df.drop(sample_df[sample_df['Name'] == 'error'].index, inplace = True) # currently dropping any gene without reads in both
    sample_df.drop(sample_df[sample_df['Name'] == 'Capsule'].index, inplace = True) # currently dropping capsule reads from plot
    sample_df.drop(sample_df[sample_df['Name'] == 'Capsule_rev'].index, inplace = True) # currently dropping capsule reads from plot
    sample_df.drop(sample_df[sample_df['Name'] == 'lacI'].index, inplace = True) # currently dropping lacI from plot
    sample_df.drop(sample_df[sample_df['Counts'] <= 100].index, inplace = True)
    sample_df = sample_df.rename(columns = {'Counts':name})
    sample_df[name] = sample_df[name]/(sum(sample_df[name])/1000000) #RPM]
    
    return sample_df


df_dir = "../../Processed Sequencing Files/230724Li/"
df_dict = dict()
names = []
# Fetch all the relevant dataframe files and put them in a useable format:
for file in os.listdir(df_dir):
    if file.endswith('_dataframe.txt'):
        name = file[:file.find('_D')]
        new_df  = pd.read_csv(''.join([df_dir ,file]))
        new_df['Length'] = new_df['Stop'] - new_df['Start']
        df_dict[name] = normalization(new_df)
        names.append(name)
print(f'\nHere is the list of unique samples for which there are dataframe files: \n\n{names}\n')


Here is the list of unique samples for which there are dataframe files: 

['Lysate_Replicate_1', 'Lysate_Replicate_2', 'Lysate_Replicate_3', 'MTC_Replicate_1', 'MTC_Replicate_2', 'MTC_Replicate_3']



#### Main Figure:
First we will generate the main figure, which is simple a scatter plot comparing the gene-by-bene RPM for 2 MTC replicates.  Later we will also generate the insert which represents the distribution of log-fold changes in RPM between the two samples. (which was overlaid over this image to create the final published image).  We will also calculate and print the Pearson coefficient (R) for the data.

In [3]:
x = 'MTC_Replicate_1' # MTC-protected RNA sample
y = 'MTC_Replicate_2' # General Lysate sample - the same as the MTC-protected RNA sample.

x_df = df_dict[x].copy()
y_df = df_dict[y].copy()
plot_df = x_df.merge(y_df)

#Now we will make a density plot based on the log RPM of each dataset:
xy = np.vstack([np.log(plot_df[x]), np.log(plot_df[y])])
plot_df['Density'] = gaussian_kde(xy)(xy)
color_mapper = LinearColorMapper(
    palette='Viridis256',
    low = min(plot_df['Density']),
    high = max(plot_df['Density']),
)

p = figure(
    y_axis_type = "log", x_axis_type = "log",
    aspect_scale = 1, width = 750, height = 750,
    output_backend = "svg", tooltips = [("Gene", "@Name")],
   # y_range = (10**1.2, 10**4.5), x_range = (10**1.2, 10**4.5)
)

p.xaxis.axis_label = "mRNA level in MTC Replicate 1 (RPM)"
p.yaxis.axis_label = "mRNA level in MTC Replicate 2 (RPM)"
p.axis.ticker = [10**0, 10**2, 10**4, 10**6]


p.circle(x=x,
         y=y,
         source=plot_df,
         size=5,
         fill_alpha=0.8,
         line_alpha=0,
         color={'field': 'Density', 'transform': color_mapper},
        )
#p.line(x=[10**(1.5), 10**4], y=[10**(1.5), 10**4], color = "black", width = 4)
colorbar = ColorBar(
    color_mapper = color_mapper,
    location = (2,4),
    title = "Gaussian Kernel Desnity",
    label_standoff = 3,
    width = 20,
    height = 500,
    bar_line_color='black',
    major_tick_line_color='black',
    major_label_text_font_size = "20px",
    title_text_font_size = "20px",
    title_text_font_style = "normal",
)
p.add_layout(colorbar, 'right')

show(p)

Calculating the Pearson Coefficient and R^2 values:

In [4]:
r = np.corrcoef(plot_df[x], plot_df[y])[0,1]
r2 = r2_score(plot_df[x], plot_df[y])
print(f"Pearson Coefficient: {r}, R^2 Value of simple linear fit: {r2}")

Pearson Coefficient: 0.9927904221104147, R^2 Value of simple linear fit: 0.9847072640154593


Run this cell in order to export figure to svg.  Note you will need to have special libraries installed for this to work:

In [5]:
p.axis.major_label_text_font = "arial"
p.axis.major_label_text_font_size = "22px"
p.axis.axis_label_text_font_style = "normal"
p.axis.axis_label_text_font = "arial"
p.axis.axis_label_text_font_size = "24px"

p.background_fill_color = "white"
p.grid.grid_line_color = None
p.title.text_color= '#000000'
p.axis.major_label_text_color = '#000000'
p.axis.axis_label_text_color = '#000000'

export_svg(p, filename = f'./1C_Main_Plot_{datetime.date.today()}.svg')

['./1C_Main_Plot_2023-09-12.svg']

#### Log Fold Distribution Insert:

Now we will generate the density plot showing the distribution of log fold changes between genes in the above plot.  We will also calculate and print the standard deviation of this distribution. 

In [6]:
plot_df['Log_Dif'] = np.log10(plot_df[x]) - np.log10(plot_df[y])

p = figure(width = 300, height = 220,
          output_backend = "svg")

# Histogram
bins = np.linspace(-1, 1, 10)
plt_hist, plt_edges = np.histogram(plot_df['Log_Dif'], density=False, bins=bins)
p.quad(top = plt_hist, bottom = 0, left = plt_edges[:-1], right = plt_edges[1:],
         fill_color="steelblue", line_color="white", fill_alpha = 1,
         )

p.yaxis.visible = False
p.yaxis.ticker = [1000, 2000]
p.xaxis.ticker = [-1, -.5, 0, .5, 1]
p.xaxis.axis_label ="log-10 ratio"

p.background_fill_color = None
p.grid.grid_line_color = None
p.grid.grid_line_width = 4
p.outline_line_color = None

show(p)

Run this cell to generate the above cell as an svg:

In [7]:
p.axis.axis_label_text_font_style = "normal"
p.axis.major_label_text_font = "arial"
p.axis.major_label_text_font_size = "22px"
p.axis.axis_label_text_font = "arial"
p.axis.axis_label_text_font_size = "22px"
p.axis.major_label_text_color = '#000000'
p.axis.axis_label_text_color = '#000000'

export_svg(p, filename = f'1C_Density_Insert_{datetime.date.today()}.svg')

['1C_Density_Insert_2023-09-12.svg']

In [8]:
std = np.std(plot_df['Log_Dif'])
print(f'The standard deviation of the log fold distribution is : {std:.3f}')

The standard deviation of the log fold distribution is : 0.067
